## RockPaperScissorsLizardSpock.js

In this notebook we will finetune a SqueezeNet pretrained on ImageNet to classify between the classes 'rock', 'paper', 'scissors', 'lizard', 'spock' and 'other'.

### Data setup

There should already be a folder for each class in `./data/train/`. E.g. all `rock` images should be in `./data/train/rock`

In [4]:
import os
import numpy as np

In [5]:
data_dir = './data/'
train_dir = data_dir + 'train/'
val_dir = data_dir + 'val/'

The output of the next cell should be (in any order) `['lizard', 'spock', 'other', 'paper', 'scissors', 'rock']`

In [6]:
categories = os.listdir(train_dir)
categories

['lizard', 'spock', 'other', 'paper', 'scissors', 'rock']

Create a train/validation split. 

In [4]:
os.makedirs(val_dir, exist_ok=True)

In [11]:
for c in categories:
    os.makedirs(val_dir + c, exist_ok=True)

In [20]:
def make_val_split(p):
    """
    Move random images of each category from train to val.
    p is the percentage of images to move, e.g.
    p == 0.2 will move 20% of each category to val.
    """
    for c in categories:
        file_names = os.listdir(train_dir + c)
        permutation = np.random.permutation(file_names)
        for i in range(int(len(file_names) * p)):
            file_path = os.path.join(c, permutation[i])
            os.rename(os.path.join(train_dir, file_path), 
                      os.path.join(val_dir, file_path))

In [21]:
# only do this once
make_val_split(0.2)

### Create and train the model

In [10]:
import numpy as np
from keras_squeezenet import SqueezeNet
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, Dropout, Activation, GlobalAveragePooling2D
from keras.models import Model

We will use SqueezeNet but replace its top layers (the classification layers) to classify between 'rock', 'paper', 'scissors', 'lizard', 'spock' and 'other' and not between the 1000 ImageNet classes.

In [8]:
base_model = SqueezeNet(weights='imagenet', include_top=False)
x = base_model.output
x = Dropout(0.5, name='drop9')(x)
x = Convolution2D(len(categories), (1, 1), padding='valid', name='conv10')(x)
x = Activation('relu', name='relu_conv10')(x)
x = GlobalAveragePooling2D()(x)

predictions = Activation('softmax', name='loss')(x)

model = Model(inputs=base_model.input, outputs=predictions)

2998272/3032184 [============================>.] - ETA: 0s

In [9]:
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

In [11]:
batch_size = 128

In [19]:
def get_batches(dirname, generator=ImageDataGenerator(), shuffle=True,
                batch_size=batch_size, class_mode='categorical', target_size=(227,227)):
    return generator.flow_from_directory(dirname, shuffle=shuffle, batch_size=batch_size,
                                 class_mode=class_mode, target_size=target_size)

In [20]:
train_batches = get_batches(train_dir)
val_batches = get_batches(val_dir)

Found 1301 images belonging to 6 classes.
Found 323 images belonging to 6 classes.


In [24]:
num_train = train_batches.samples
num_val = val_batches.samples

In [27]:
def fit(model, epochs):
    model.fit_generator(train_batches, 
                        steps_per_epoch=num_train // batch_size,
                        epochs=epochs,
                        validation_data=val_batches,
                        validation_steps=num_val // batch_size)

In [25]:
fit(model, 5)

Epoch 1/5
10/10 [==============================] - 49s - loss: 1.9781 - acc: 0.3270 - val_loss: 1.5053 - val_acc: 0.4258
Epoch 2/5
10/10 [==============================] - 48s - loss: 1.5591 - acc: 0.3338 - val_loss: 1.5009 - val_acc: 0.2539
Epoch 3/5
10/10 [==============================] - 48s - loss: 1.4380 - acc: 0.3636 - val_loss: 1.5426 - val_acc: 0.3047
Epoch 4/5
10/10 [==============================] - 48s - loss: 1.5838 - acc: 0.2872 - val_loss: 1.5208 - val_acc: 0.4023
Epoch 5/5
10/10 [==============================] - 48s - loss: 1.4855 - acc: 0.3527 - val_loss: 1.4059 - val_acc: 0.4922


In [28]:
model.optimizer.lr = 0.1
fit(model, 2)

Epoch 1/2
10/10 [==============================] - 53s - loss: 1.3813 - acc: 0.4422 - val_loss: 1.2947 - val_acc: 0.4727
Epoch 2/2
10/10 [==============================] - 44s - loss: 1.3131 - acc: 0.4481 - val_loss: 1.1375 - val_acc: 0.5195


In [29]:
model.optimizer.lr = 0.01
fit(model, 1)

Epoch 1/1
10/10 [==============================] - 49s - loss: 1.2365 - acc: 0.4653 - val_loss: 1.5440 - val_acc: 0.4102


In [30]:
model.optimizer.lr = 0.0001
fit(model, 3)

Epoch 1/3
10/10 [==============================] - 49s - loss: 1.3378 - acc: 0.4472 - val_loss: 1.1216 - val_acc: 0.6250
Epoch 2/3
10/10 [==============================] - 48s - loss: 1.1095 - acc: 0.5644 - val_loss: 0.8856 - val_acc: 0.6484
Epoch 3/3
10/10 [==============================] - 48s - loss: 0.8915 - acc: 0.6411 - val_loss: 0.9533 - val_acc: 0.5586


In [31]:
model.optimizer.lr = 1e-5
fit(model, 3)

Epoch 1/3
10/10 [==============================] - 49s - loss: 0.8998 - acc: 0.6612 - val_loss: 0.7066 - val_acc: 0.7656
Epoch 2/3
10/10 [==============================] - 52s - loss: 0.5834 - acc: 0.7977 - val_loss: 0.4073 - val_acc: 0.9062
Epoch 3/3
10/10 [==============================] - 44s - loss: 1.2958 - acc: 0.6536 - val_loss: 1.1508 - val_acc: 0.5234


In [32]:
model.optimizer.lr = 1e-6
fit(model, 3)

Epoch 1/3
10/10 [==============================] - 49s - loss: 1.0495 - acc: 0.6164 - val_loss: 0.7838 - val_acc: 0.7812
Epoch 2/3
10/10 [==============================] - 48s - loss: 0.7439 - acc: 0.7311 - val_loss: 0.7127 - val_acc: 0.7539
Epoch 3/3
10/10 [==============================] - 52s - loss: 0.6578 - acc: 0.7625 - val_loss: 0.5134 - val_acc: 0.8281


In [ ]:
fit(model, 10)

Epoch 1/10
10/10 [==============================] - 49s - loss: 0.3665 - acc: 0.8905 - val_loss: 0.2443 - val_acc: 0.9297
Epoch 2/10
10/10 [==============================] - 48s - loss: 0.2053 - acc: 0.9223 - val_loss: 0.0870 - val_acc: 0.9727
Epoch 3/10
 1/10 [==>...........................] - ETA: 44s - loss: 0.0871 - acc: 0.9688

In [34]:
model.save_weights('model.hdf5')

In [39]:
import tensorflow as tf
from keras import backend as K

In [41]:
session = K.get_session()
saver = tf.train.Saver()
saver.save(session, 'model.ckpt')

'model.ckpt'

In [42]:
%run dump_checkpoint_vars.py --output-dir=deeplearn-checkpoint --checkpoint-file=data/models/model.ckpt

usage: dump_checkpoint_vars.py [-h] --checkpoint_file CHECKPOINT_FILE
                               --output_dir OUTPUT_DIR
                               [--remove_variables_regex REMOVE_VARIABLES_REGEX]
dump_checkpoint_vars.py: error: the following arguments are required: --checkpoint_file, --output_dir


SystemExit: 2